In [2]:
import os
from pathlib import Path, PureWindowsPath
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pearsonr
from scipy.stats import ttest_ind
import statsmodels.api as sm
from statsmodels.formula.api import ols
import statsmodels.api as sm

#Packages related to clustering
from tslearn.clustering import silhouette_score
from sklearn.decomposition import PCA
from tslearn.clustering import TimeSeriesKMeans
from tslearn.datasets import CachedDatasets
from tslearn.preprocessing import TimeSeriesScalerMeanVariance
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [1]:
pip install tslearn

Note: you may need to restart the kernel to use updated packages.


In [130]:
os.path.dirname(os.getcwd())

'C:\\Users\\Cornelius\\Cory Dropbox\\Cory LeRoy\\PC\\Documents\\GitHub\\Store-Sales'

In [131]:
path_cwd = Path(PureWindowsPath(os.path.dirname(os.getcwd())))
path_cwd

WindowsPath('C:/Users/Cornelius/Cory Dropbox/Cory LeRoy/PC/Documents/GitHub/Store-Sales')

In [132]:
path = path_cwd / 'data'
path

WindowsPath('C:/Users/Cornelius/Cory Dropbox/Cory LeRoy/PC/Documents/GitHub/Store-Sales/data')

In [188]:
df_train = pd.read_csv(path / 'train.csv')
df_transaction = pd.read_csv(path / 'transactions.csv')
df_holidays = pd.read_csv(path / 'holidays_events.csv')
df_oil = pd.read_csv(path / 'oil.csv')
df_stores = pd.read_csv(path / 'stores.csv')
df_test = pd.read_csv(path / 'test.csv')

In [169]:
import os

# Importing for Will

csv_dir = '/Users/williamearley/Documents/Store-Sales/data'  

# Construct the file paths using os.path.join()
csv_train_path = os.path.join(csv_dir, 'train.csv')
csv_transaction_path = os.path.join(csv_dir, 'transactions.csv')
csv_holidays_path = os.path.join(csv_dir, 'holidays_events.csv')
csv_oil = os.path.join(csv_dir, 'oil.csv')
csv_stores = os.path.join(csv_dir, 'stores.csv')
csv_test = os.path.join(csv_dir, 'test.csv')

# Now, read the CSV files
df_train = pd.read_csv(csv_train_path)
df_transaction = pd.read_csv(csv_transaction_path)
df_holidays = pd.read_csv(csv_holidays_path)
df_oil = pd.read_csv(csv_oil)
df_stores = pd.read_csv(csv_stores)
df_test = pd.read_csv(csv_test)


## Clean Data ##

In [170]:
#rename oil to be used for missing value interpolation
new_oil = df_oil.copy()
new_oil.rename(columns={'dcoilwtico': 'oil_price'}, inplace=True)

In [171]:
# Assuming df is your DataFrame and 'column_name' is the name of the column
null_count = new_oil['oil_price'].isnull().sum()

# This will give you the count of null values in the 'column_name' column
print("Count of null values in 'oil_price':", null_count)

Count of null values in 'oil_price': 43


In [172]:
new_oil['oil_price'] = new_oil['oil_price'].interpolate(method='index')

In [173]:
# Assuming df is your DataFrame and 'column_name' is the name of the column
null_count = new_oil['oil_price'].isnull().sum()

# This will give you the count of null values in the 'column_name' column
print("Count of null values in 'oil_price':", null_count)

Count of null values in 'oil_price': 1


In [174]:
new_oil.at[0, 'oil_price'] = 93.14

In [175]:
# Assuming df is your DataFrame and 'column_name' is the name of the column
null_count = new_oil['oil_price'].isnull().sum()

# This will give you the count of null values in the 'column_name' column
print("Count of null values in 'oil_price':", null_count)

Count of null values in 'oil_price': 0


#### shorten date ####

In [176]:
def train_to_store_merge(train, store):
    df_train_store_merged = pd.merge(train, store, how='left', on='store_nbr')
    return df_train_store_merged

In [177]:
# train clean

df_train['date'] = pd.to_datetime(df_train['date'])
first_sale_date_per_store = df_train[df_train['sales'] > 0].groupby('store_nbr')['date'].min().reset_index()

# remove rows before stores were open. only do this to train
df_train_min_date = pd.merge(df_train, first_sale_date_per_store, on='store_nbr')
df_train_shortened = df_train_min_date[df_train_min_date['date_x'] >= df_train_min_date['date_y']] 
df_train_shortened = df_train_shortened.drop(['date_y'], axis=1)
df_train_shortened.rename(columns={'date_x':'date'}, inplace=True)
df_train_shortened = train_to_store_merge(df_train_shortened, df_stores)

# remove dates when stores were temporarily closed
df_train_short = df_train_shortened[~((df_train_shortened['date']>='2016-08-22') &(df_train_shortened['date']<='2016-10-26') &(df_train_shortened['store_nbr']==25))]
df_train_short =df_train_short[~((df_train_short['date']>='2014-04-14') &(df_train_short['date']<='2014-07-23') &(df_train_short['store_nbr']==24))]
df_train_short =df_train_short[~((df_train_short['date']>='2013-07-08') &(df_train_short['date']<='2013-07-30') &(df_train_short['store_nbr']==30))]
df_train_short =df_train_short[~((df_train_short['date']>='2014-08-04') &(df_train_short['date']<='2014-09-10') &(df_train_short['store_nbr']==14))]
df_train_short =df_train_short[~((df_train_short['date']>='2015-03-30') &(df_train_short['date']<='2015-05-28') &(df_train_short['store_nbr']==12))]
df_train_shortened =df_train_short[~((df_train_short['date']>='2016-08-15') &(df_train_short['date']<='2016-12-02') &(df_train_short['store_nbr']==18))]

In [178]:
def create_date_features(df):
    df_train['date'] = pd.to_datetime(df_train['date'])
    df['day_of_week'] = df['date'].dt.dayofweek
    df['month'] = df['date'].dt.month
    df['year'] = df['date'].dt.year
    return df

#### holidays feature eng ####

In [179]:
# holidays 

df_holidays['date'] = pd.to_datetime(df_holidays['date'])
df_holidays_real = df_holidays[df_holidays['transferred']==False] 

#unique_holis2 = df_holidays_real['description'].drop_duplicates()
unique_holis2 = df_holidays_real[['description','locale']].drop_duplicates()
unique_holis_national = unique_holis2[unique_holis2['locale']=='National'].drop(['locale'],axis=1)
unique_holis_city = unique_holis2[unique_holis2['locale']=='Local'].drop(['locale'],axis=1)
unique_holis_state = unique_holis2[unique_holis2['locale']=='Regional'].drop(['locale'],axis=1)

national_holidays = df_holidays_real[df_holidays_real['locale']=='National'].loc[:,('date','description')]
local_holidays = df_holidays_real[df_holidays_real['locale']=='Local'].loc[:,('date','description','locale_name')]
state_holidays = df_holidays_real[df_holidays_real['locale']=='Regional'].loc[:,('date','description','locale_name')]

df_train_summed_daily = df_train_shortened.groupby(['date','city','state']).agg({'onpromotion':'sum', 'sales':'sum'}).reset_index()


In [180]:
df_holi = pd.merge(df_train_summed_daily, national_holidays, how='left', on='date')
df_holi = pd.merge(df_holi, state_holidays, how='left', left_on=['date', 'state'], right_on=['date','locale_name'])
df_holi = pd.merge(df_holi, local_holidays, how='left', left_on=['date', 'city'], right_on=['date','locale_name'])

df_holi = df_holi.drop(['locale_name_x','locale_name_y'],axis=1)
df_holi = df_holi.rename(columns = {'description_x':'national_holiday','description_y':'state_holiday','description':'city_holiday'})

In [181]:
# set boolean column for each unique holiday. still have dups

for holiday in unique_holis_national['description'].tolist():
    df_holi[holiday] = df_holi['national_holiday'] == holiday
for holiday in unique_holis_state['description'].tolist():
    df_holi[holiday] = df_holi['state_holiday'] == holiday
for holiday in unique_holis_city['description'].tolist():
    df_holi[holiday] = df_holi['city_holiday'] == holiday

df_holi = df_holi.drop(['national_holiday','state_holiday','city_holiday'] ,axis=1)

/var/folders/25/ys3kvvh94qz9hdnmxh9q884h0000gn/T/ipykernel_8599/194147805.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_holi[holiday] = df_holi['city_holiday'] == holiday
/var/folders/25/ys3kvvh94qz9hdnmxh9q884h0000gn/T/ipykernel_8599/194147805.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_holi[holiday] = df_holi['city_holiday'] == holiday
/var/folders/25/ys3kvvh94qz9hdnmxh9q884h0000gn/T/ipykernel_8599/194147805.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `fr

In [182]:
# combines duplicates rows into 1 rows where there is a true for each holiday that falls on the given date
unique_holis_list = list(unique_holis2['description'])
agg_func = {col: 'any' for col in unique_holis_list}

aggregated_df = df_holi.groupby(['date', 'sales', 'city', 'state', 'onpromotion']).agg(agg_func).reset_index()


aggregated_df = pd.get_dummies(aggregated_df, columns=['city', 'state'], prefix=['city', 'state'])

In [183]:
#OLS to determine impact of each holiday across all stores

In [184]:
# seasonality has big impact on predicted sales so need to add some variables to capture 
aggregated_df_ols = aggregated_df.copy()
aggregated_df_ols['dow'] = aggregated_df_ols['date'].dt.dayofweek
aggregated_df_ols['month'] = aggregated_df_ols['date'].dt.month
aggregated_df_ols['year'] = aggregated_df_ols['date'].dt.year
aggregated_df_ols = aggregated_df_ols.drop('date',axis=1)

In [185]:
#model seasonality with dummy vars... do i need this?
aggregated_df_ols_dummies =  pd.get_dummies(aggregated_df_ols, columns=['dow','month', 'year'], drop_first=True)

#### add oil interpolated, lag_1, rolling mean ####

In [186]:
def add_lag_rolling(aggregated_df_ols_dummies):
        
# Lag sales by 1 to get previous day of sales value
    aggregated_df_ols_dummies['lag_1'] = aggregated_df_ols_dummies['sales'].shift(1)

# Add rolling mean for 7-day window
    aggregated_df_ols_dummies['rolling_mean'] = aggregated_df_ols_dummies['sales'].rolling(window=7).mean()


    aggregated_df_ols_dummies['lag_1'].fillna(aggregated_df_ols_dummies['sales'], inplace=True)
    aggregated_df_ols_dummies['rolling_mean'].fillna(aggregated_df_ols_dummies['sales'],inplace=True)
    return aggregated_df_ols_dummies

In [187]:
aggregated_df_ols_dummies = add_lag_rolling(aggregated_df_ols_dummies)

#### Further Modeling ####

In [188]:
X = aggregated_df_ols_dummies.drop('sales',axis=1)
# add constant for linear regression
X = sm.add_constant(X)
X = X.astype(int)
y=aggregated_df_ols_dummies['sales']

In [189]:
model_OLS = sm.OLS(y,X).fit()
model_summary = model_OLS.summary()

In [190]:
# remove features that have p value >.10 
# ... no xmas day in trainset
pvalues = model_OLS.pvalues
alpha = .05
drop_columns = pvalues.to_frame(name='feature').reset_index()
drop_columns = drop_columns[(~drop_columns['index'].str.contains('city')) \
                            & (~drop_columns['index'].str.contains('month')) \
                            & (~drop_columns['index'].str.contains('year')) \
                            & (~drop_columns['index'].str.contains('state')) \
                            & (drop_columns['feature']>alpha) \
                           ]
#drop_columns = pvalues[(pvalues>alpha)]
drop_cols = list(drop_columns['index'])
drop_cols

['Fundacion de Manta',
 'Provincializacion de Cotopaxi',
 'Fundacion de Cuenca',
 'Cantonizacion de Libertad',
 'Cantonizacion de Riobamba',
 'Cantonizacion del Puyo',
 'Cantonizacion de Guaranda',
 'Provincializacion de Imbabura',
 'Cantonizacion de Latacunga',
 'Fundacion de Machala',
 'Fundacion de Santo Domingo',
 'Cantonizacion de El Carmen',
 'Cantonizacion de Cayambe',
 'Fundacion de Esmeraldas',
 'Primer Grito de Independencia',
 'Fundacion de Riobamba',
 'Fundacion de Ambato',
 'Fundacion de Ibarra',
 'Cantonizacion de Quevedo',
 'Traslado Independencia de Guayaquil',
 'Dia de Difuntos',
 'Provincializacion de Santo Domingo',
 'Provincializacion Santa Elena',
 'Independencia de Guaranda',
 'Independencia de Latacunga',
 'Independencia de Ambato',
 'Fundacion de Quito-1',
 'Fundacion de Loja',
 'Puente Navidad',
 'Navidad',
 'Navidad+1',
 'Puente Primer dia del ano',
 'Primer dia del ano-1',
 'Recupero puente Navidad',
 'Recupero puente primer dia del ano',
 'Viernes Santo',
 '

In [191]:
#values2 = pd.DataFrame(pvalues).reset_index()
#pvalues2.rename(columns={'index':'holiday',0:'p'} , inplace=True)
#pvalues2

In [192]:
print(len(drop_cols), 'holidays dropped out of', len(unique_holis2),'.', len(unique_holis2)-len(drop_cols) ,'unique holidays remain' )

85 holidays dropped out of 103 . 18 unique holidays remain


In [193]:
df_holi_shortened = df_holidays_real[~df_holidays_real['description'].isin(drop_cols)]
df_holi_shortened = df_holi_shortened.drop(['type','locale','locale_name','transferred'],axis=1)

In [194]:
# unique holidays from the reduced holidays list
unique_holidays = df_holi_shortened['description'].unique()
filtered_holidays = df_holidays[df_holidays['description'].isin(unique_holidays)]

In [195]:
#separate these out because they have differenct merge conditions
national_holidays_filtered = filtered_holidays[filtered_holidays['locale']=='National'].loc[:,('date','description')]
local_holidays_filtered = filtered_holidays[filtered_holidays['locale']=='Local'].loc[:,('date','description','locale_name')]
state_holidays_filtered = filtered_holidays[filtered_holidays['locale']=='Regional'].loc[:,('date','description','locale_name')]

In [196]:
# unique holidays from the reduced holidays list
unique_holidays = df_holi_shortened['description'].unique()

#### merge train to holidays ####

In [197]:
# unique holidays from the reduced holidays list
filtered_holidays = df_holidays[df_holidays['description'].isin(unique_holidays)]
#separate locales out because they have differenct merge conditions
national_holidays_filtered = filtered_holidays[filtered_holidays['locale']=='National'].loc[:,('date','description')]
local_holidays_filtered = filtered_holidays[filtered_holidays['locale']=='Local'].loc[:,('date','description','locale_name')]
state_holidays_filtered = filtered_holidays[filtered_holidays['locale']=='Regional'].loc[:,('date','description','locale_name')]

In [198]:
local_holidays_filtered

,date,description,locale_name
29,2012-12-06,Fundacion de Quito,Quito
32,2012-12-22,Cantonizacion de Salinas,Salinas
82,2013-12-06,Fundacion de Quito,Quito
86,2013-12-22,Cantonizacion de Salinas,Salinas
147,2014-12-06,Fundacion de Quito,Quito
151,2014-12-22,Cantonizacion de Salinas,Salinas
201,2015-12-06,Fundacion de Quito,Quito
205,2015-12-22,Cantonizacion de Salinas,Salinas
287,2016-12-06,Fundacion de Quito,Quito
291,2016-12-22,Cantonizacion de Salinas,Salinas


In [199]:
def train_to_holiday_merge(train, national_holidays_filtered, state_holidays_filtered, local_holidays_filtered):
    #train['date'] = pd.to_datetime(train['date'])
    df_train_filtered = pd.merge(train, national_holidays_filtered, how='left', on='date')
    df_train_filtered = pd.merge(df_train_filtered, state_holidays_filtered, how='left', left_on=['date', 'state'], right_on=['date','locale_name'])
    df_train_filtered = pd.merge(df_train_filtered, local_holidays_filtered, how='left', left_on=['date', 'city'], right_on=['date','locale_name'])
    df_train_filtered['holiday'] = df_train_filtered['description_x'].combine_first(df_train_filtered['description_y']).combine_first(df_train_filtered['description'])

    df_train_filtered = df_train_filtered.drop(['locale_name_x','locale_name_y','description','description_x','description_y'],axis=1)
    return df_train_filtered

In [200]:
def train_to_oil_merge(train, oil):
    oil['date'] = pd.to_datetime(oil['date'])
    df = pd.merge(train,oil, how='left', on='date')
    return df

### K-Means Clustering

In [201]:
df_daily_sales_by_family = df_train_shortened.groupby(['date', 'family'])['sales'].sum().reset_index()

In [202]:
# Convert date column to datetime if it's not already in datetime format
df_daily_sales_by_family['date'] = pd.to_datetime(df_daily_sales_by_family['date'])

# Pivot the data
df_pivot = df_daily_sales_by_family.pivot(index='date', columns='family', values='sales').fillna(0)

df_pivot = df_pivot.T

XTrain = df_pivot.to_numpy()
XCategories = df_pivot.index

seed = 0
np.random.seed(seed)
x_train = TimeSeriesScalerMeanVariance().fit_transform(XTrain)
sz=x_train.shape[1]

n_clusters = 6
sz = x_train.shape[1]

km = TimeSeriesKMeans(n_clusters=n_clusters, verbose=False, random_state=seed)
y_pred_km = km.fit_predict(x_train)

myDict = {}

for i in range(len(XCategories)):
    key = XCategories[i]
    value = y_pred_km[i]
    myDict[key] = value


def familyCluster(fam):
    return myDict.get(fam)

df_train_shortened['familycluster'] = df_train_shortened['family'].apply(lambda x: familyCluster(x))

/var/folders/25/ys3kvvh94qz9hdnmxh9q884h0000gn/T/ipykernel_8599/3592288157.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_shortened['familycluster'] = df_train_shortened['family'].apply(lambda x: familyCluster(x))


In [203]:
df_train_filtered = train_to_holiday_merge(df_train_shortened, national_holidays_filtered, state_holidays_filtered,local_holidays_filtered)
df_train_filtered

,id,date,store_nbr,family,sales,onpromotion,city,state,type,cluster,familycluster,holiday
0,1782,2013-01-02,1,AUTOMOTIVE,2.000,0,Quito,Pichincha,D,13,5,NaN
1,1783,2013-01-02,1,BABY CARE,0.000,0,Quito,Pichincha,D,13,0,NaN
2,1784,2013-01-02,1,BEAUTY,2.000,0,Quito,Pichincha,D,13,5,NaN
3,1785,2013-01-02,1,BEVERAGES,1091.000,0,Quito,Pichincha,D,13,0,NaN
4,1786,2013-01-02,1,BOOKS,0.000,0,Quito,Pichincha,D,13,5,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2765692,3000883,2017-08-15,9,POULTRY,438.133,0,Quito,Pichincha,B,6,3,NaN
2765693,3000884,2017-08-15,9,PREPARED FOODS,154.553,1,Quito,Pichincha,B,6,3,NaN
2765694,3000885,2017-08-15,9,PRODUCE,2419.729,148,Quito,Pichincha,B,6,0,NaN
2765695,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,Quito,Pichincha,B,6,5,NaN


In [204]:
#### combine train ####
df_train_filtered = train_to_holiday_merge(df_train_shortened, national_holidays_filtered, state_holidays_filtered,local_holidays_filtered)
df_train_filtered = create_date_features(df_train_filtered)
df_train_merged = train_to_oil_merge(df_train_filtered, new_oil)

df_train_merged = df_train_merged.drop(['id','city','state', 'type'], axis=1)


In [205]:
df_train_merged[df_train_merged['oil_price'].isna()]

,date,store_nbr,family,sales,onpromotion,cluster,familycluster,holiday,day_of_week,month,year,oil_price
99,2013-01-05,1,AUTOMOTIVE,5.000,0,13,5,NaN,5,1,2013,NaN
100,2013-01-05,1,BABY CARE,0.000,0,13,0,NaN,5,1,2013,NaN
101,2013-01-05,1,BEAUTY,3.000,0,13,5,NaN,5,1,2013,NaN
102,2013-01-05,1,BEVERAGES,1160.000,0,13,0,NaN,5,1,2013,NaN
103,2013-01-05,1,BOOKS,0.000,0,13,5,NaN,5,1,2013,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2765626,2017-08-13,9,POULTRY,412.458,0,6,3,NaN,6,8,2017,NaN
2765627,2017-08-13,9,PREPARED FOODS,105.169,1,6,3,NaN,6,8,2017,NaN
2765628,2017-08-13,9,PRODUCE,1693.607,7,6,0,NaN,6,8,2017,NaN
2765629,2017-08-13,9,SCHOOL AND OFFICE SUPPLIES,200.000,8,6,5,NaN,6,8,2017,NaN


In [206]:
# df_oil is completely missing some days. after left joining, need to interpolate again
df_train_merged['oil_price'] = df_train_merged['oil_price'].interpolate(method='index')
#new_oil['oil_price'] = new_oil['oil_price'].interpolate(method='index')

In [207]:
df_train_dummies = pd.get_dummies(df_train_merged, columns=['cluster','holiday','familycluster']) ## add family pca here maybe?
# if we arent getting expected results, create dummies for date features
df_train_dummies.drop(columns=['family','store_nbr'], inplace=True)
df_train_dummies

,date,sales,onpromotion,day_of_week,month,year,oil_price,cluster_1,cluster_2,cluster_3,...,holiday_Terremoto Manabi+3,holiday_Terremoto Manabi+4,holiday_Terremoto Manabi+5,holiday_Traslado Primer dia del ano,familycluster_0,familycluster_1,familycluster_2,familycluster_3,familycluster_4,familycluster_5
0,2013-01-02,2.000,0,2,1,2013,93.14,False,False,False,...,False,False,False,False,False,False,False,False,False,True
1,2013-01-02,0.000,0,2,1,2013,93.14,False,False,False,...,False,False,False,False,True,False,False,False,False,False
2,2013-01-02,2.000,0,2,1,2013,93.14,False,False,False,...,False,False,False,False,False,False,False,False,False,True
3,2013-01-02,1091.000,0,2,1,2013,93.14,False,False,False,...,False,False,False,False,True,False,False,False,False,False
4,2013-01-02,0.000,0,2,1,2013,93.14,False,False,False,...,False,False,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2765692,2017-08-15,438.133,0,1,8,2017,47.57,False,False,False,...,False,False,False,False,False,False,False,True,False,False
2765693,2017-08-15,154.553,1,1,8,2017,47.57,False,False,False,...,False,False,False,False,False,False,False,True,False,False
2765694,2017-08-15,2419.729,148,1,8,2017,47.57,False,False,False,...,False,False,False,False,True,False,False,False,False,False
2765695,2017-08-15,121.000,8,1,8,2017,47.57,False,False,False,...,False,False,False,False,False,False,False,False,False,True


In [208]:
df_train_dummies.shape

(2765697, 47)

In [209]:
dummy_columns_train = df_train_dummies.columns

In [210]:
columns_to_scale =  list(df_train_dummies.columns)


In [212]:
columns_to_scale.remove('sales')

In [166]:
del columns_to_scale[0]

In [213]:
columns_to_scale

['onpromotion',
 'day_of_week',
 'month',
 'year',
 'oil_price',
 'cluster_1',
 'cluster_2',
 'cluster_3',
 'cluster_4',
 'cluster_5',
 'cluster_6',
 'cluster_7',
 'cluster_8',
 'cluster_9',
 'cluster_10',
 'cluster_11',
 'cluster_12',
 'cluster_13',
 'cluster_14',
 'cluster_15',
 'cluster_16',
 'cluster_17',
 'holiday_Carnaval',
 'holiday_Dia de la Madre',
 'holiday_Dia del Trabajo',
 'holiday_Fundacion de Quito',
 'holiday_Independencia de Cuenca',
 'holiday_Mundial de futbol Brasil: Ecuador-Suiza',
 'holiday_Navidad-1',
 'holiday_Navidad-2',
 'holiday_Navidad-3',
 'holiday_Navidad-4',
 'holiday_Primer dia del ano',
 'holiday_Terremoto Manabi+1',
 'holiday_Terremoto Manabi+2',
 'holiday_Terremoto Manabi+3',
 'holiday_Terremoto Manabi+4',
 'holiday_Terremoto Manabi+5',
 'holiday_Traslado Primer dia del ano',
 'familycluster_0',
 'familycluster_1',
 'familycluster_2',
 'familycluster_3',
 'familycluster_4',
 'familycluster_5']

In [214]:
# Introduce MinMaxScaler
scaler = MinMaxScaler()

# Apply
df_train_dummies[columns_to_scale] = scaler.fit_transform(df_train_dummies[columns_to_scale])
df_train_dummies

,date,sales,onpromotion,day_of_week,month,year,oil_price,cluster_1,cluster_2,cluster_3,...,holiday_Terremoto Manabi+3,holiday_Terremoto Manabi+4,holiday_Terremoto Manabi+5,holiday_Traslado Primer dia del ano,familycluster_0,familycluster_1,familycluster_2,familycluster_3,familycluster_4,familycluster_5
0,2013-01-02,2.000,0.000000,0.333333,0.000000,0.0,0.792965,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,2013-01-02,0.000,0.000000,0.333333,0.000000,0.0,0.792965,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,2013-01-02,2.000,0.000000,0.333333,0.000000,0.0,0.792965,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,2013-01-02,1091.000,0.000000,0.333333,0.000000,0.0,0.792965,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,2013-01-02,0.000,0.000000,0.333333,0.000000,0.0,0.792965,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2765692,2017-08-15,438.133,0.000000,0.166667,0.636364,1.0,0.253228,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2765693,2017-08-15,154.553,0.001350,0.166667,0.636364,1.0,0.253228,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2765694,2017-08-15,2419.729,0.199730,0.166667,0.636364,1.0,0.253228,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2765695,2017-08-15,121.000,0.010796,0.166667,0.636364,1.0,0.253228,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [215]:
#df_train_dummies[df_train_dummies.isnull()]
rows_with_na = df_train_dummies[df_train_dummies.isna().any(axis=1)]
rows_with_na

,date,sales,onpromotion,day_of_week,month,year,oil_price,cluster_1,cluster_2,cluster_3,...,holiday_Terremoto Manabi+3,holiday_Terremoto Manabi+4,holiday_Terremoto Manabi+5,holiday_Traslado Primer dia del ano,familycluster_0,familycluster_1,familycluster_2,familycluster_3,familycluster_4,familycluster_5


In [47]:
print(df_train_dummies.shape, df_train.shape)


(2765697, 47) (3000888, 6)


In [216]:
df_train_dummies.to_pickle('df_train_sales_no_scale.pkl')

## Transformations on test data ##

In [49]:
df_test_transformed = train_to_store_merge(df_test, df_stores)
df_test_transformed['date'] = pd.to_datetime(df_test_transformed['date'])
df_test_transformed = create_date_features(df_test_transformed)
df_test_transformed = train_to_holiday_merge(df_test_transformed, national_holidays_filtered, state_holidays_filtered, local_holidays_filtered)

df_test_transformed = train_to_oil_merge(df_test_transformed, new_oil)
#have to then re interpolate after the merge
df_test_transformed['oil_price'] = df_test_transformed['oil_price'].interpolate(method='index')

df_test_transformed = df_test_transformed.drop(['id','city','state', 'type'], axis=1)
df_test_transformed = create_date_features(df_test_transformed)

df_test_transformed['familycluster'] = df_test_transformed['family'].apply(lambda x: familyCluster(x))

df_test_transformed.drop(columns=['family', 'store_nbr'],inplace=True)
df_test_transformed.shape

(28512, 9)

In [50]:
df_test_dummies = pd.get_dummies(df_test_transformed, columns=['cluster','holiday','familycluster']) ## add family pca here maybe?
df_test_dummies.columns
df_test_dummies = df_test_dummies.reindex(columns=dummy_columns_train)

In [51]:
df_test_dummies.fillna(0, inplace=True)
columns_to_scale =  list(df_test_dummies.columns)
columns_to_scale.remove('date')
# Introduce MinMaxScaler
scaler = MinMaxScaler()

# Apply
df_test_dummies[columns_to_scale] = scaler.fit_transform(df_test_dummies[columns_to_scale])
df_test_dummies

,date,sales,onpromotion,day_of_week,month,year,oil_price,cluster_1,cluster_2,cluster_3,...,holiday_Terremoto Manabi+3,holiday_Terremoto Manabi+4,holiday_Terremoto Manabi+5,holiday_Traslado Primer dia del ano,familycluster_0,familycluster_1,familycluster_2,familycluster_3,familycluster_4,familycluster_5
0,2017-08-16,0.0,0.000000,0.333333,0.0,0.0,0.319392,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,2017-08-16,0.0,0.000000,0.333333,0.0,0.0,0.319392,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,2017-08-16,0.0,0.003096,0.333333,0.0,0.0,0.319392,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,2017-08-16,0.0,0.030960,0.333333,0.0,0.0,0.319392,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,2017-08-16,0.0,0.000000,0.333333,0.0,0.0,0.319392,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28507,2017-08-31,0.0,0.001548,0.500000,0.0,0.0,0.494297,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
28508,2017-08-31,0.0,0.000000,0.500000,0.0,0.0,0.494297,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
28509,2017-08-31,0.0,0.001548,0.500000,0.0,0.0,0.494297,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
28510,2017-08-31,0.0,0.013932,0.500000,0.0,0.0,0.494297,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [52]:
rows_with_na = df_test_dummies[df_test_dummies.isna().any(axis=1)]
rows_with_na
# no longer any rows with NaN or NaT

,date,sales,onpromotion,day_of_week,month,year,oil_price,cluster_1,cluster_2,cluster_3,...,holiday_Terremoto Manabi+3,holiday_Terremoto Manabi+4,holiday_Terremoto Manabi+5,holiday_Traslado Primer dia del ano,familycluster_0,familycluster_1,familycluster_2,familycluster_3,familycluster_4,familycluster_5


In [53]:
df_test_dummies.to_pickle('df_test.pkl')

In [55]:
df_train = pd.read_pickle('df_train.pkl')

In [129]:
df_test = pd.read_pickle('df_test.pkl')

In [56]:
df_train.shape

(2765697, 47)

In [57]:
df_test.shape

(28512, 47)

In [98]:
df_train.drop(columns=['date'], inplace=True)

In [99]:
df_train.head(3)

,sales,onpromotion,day_of_week,month,year,oil_price,cluster_1,cluster_2,cluster_3,cluster_4,...,holiday_Terremoto Manabi+3,holiday_Terremoto Manabi+4,holiday_Terremoto Manabi+5,holiday_Traslado Primer dia del ano,familycluster_0,familycluster_1,familycluster_2,familycluster_3,familycluster_4,familycluster_5
0,0.000016,0.0,0.333333,0.0,0.0,0.792965,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.000000,0.0,0.333333,0.0,0.0,0.792965,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.000016,0.0,0.333333,0.0,0.0,0.792965,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


# Modeling with XGBoost

In [100]:
# Machine Learning Modeling
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import RandomizedSearchCV
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import joblib
from sklearn.pipeline import Pipeline

In [121]:
# Define a function to compute the evaluations metrics after the forecast
def evaluate_forecast(y_test, forecast):
    """
    Compute MSE, RMSE, and RMSLE for a forecast.

    Parameters:
    y_test (array-like): Actual values.
    forecast (array-like): Predicted values.

    Returns:
    dict: Dictionary containing MSE, RMSE, and RMSLE.
    """
    def rmsle(predicted, actual):
        return np.sqrt(np.mean(np.square(np.log1p(predicted) - np.log1p(actual))))

    # Compute Mean Squared Error (MSE)
    mse = mean_squared_error(y_test, forecast)
    
    # Compute Root Mean Squared Error (RMSE)
    rmse = np.sqrt(mse)
    
    # Compute Root Mean Squared Logarithmic Error (RMSLE)
    rmsle_value = rmsle(forecast, y_test)
    
    # Return the evaluation metrics as a dictionary
    metrics = {
        'MSE': mse,
        'RMSE': rmse,
        'RMSLE': rmsle_value
    }
    
    return metrics

In [101]:
train_size_feat = int(0.8 * len(df_train))
train_data_feat = df_train[:train_size_feat]
test_data_feat = df_train[train_size_feat:]

In [102]:
X_train_feat = train_data_feat.drop(columns=['sales'])
y_train_feat = train_data_feat['sales']

In [103]:
X_test_feat = test_data_feat.drop(columns=['sales'])
y_test_feat = test_data_feat['sales']

In [104]:
# Params
xgb_param_dist = {
    'learning_rate': [0.01, 0.1, 0.3],
    'n_estimators': [50, 100, 200],
    'max_depth': [2, 4, 6]
}

# Model Instance
model_xgb = xgb.XGBRegressor(objective='reg:squarederror')

# Need to enable categorical encoding for the 'date' column
xgb_params_with_categorical = xgb_param_dist.copy()
xgb_params_with_categorical['enable_categorical'] = [True]

# Perform the search
random_search = RandomizedSearchCV(
    model_xgb,
    param_distributions=xgb_param_dist,
    n_iter=10,  # Number of iterations as needed
    cv=5,       # Number of cross-validation folds
    scoring='neg_mean_squared_error',
    random_state=42
)

# Fit the model
random_search.fit(X_train_feat, y_train_feat)

# Get the best params
best_xgb_params = random_search.best_params_

# Print out
print("Best XGBoost parameters:", best_xgb_params)

# Get best estimator
best_xgb_model = random_search.best_estimator_

# Print out
best_xgb_model

Best XGBoost parameters: {'n_estimators': 200, 'max_depth': 6, 'learning_rate': 0.1}


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=6, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=200, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

To try tomorrow, maybe drop date column before feeding into model, and then put it back after?

In [123]:
forecast_xgb_best = best_xgb_model.predict(X_test_feat)

In [124]:
# Calculate Evaluation Metrics
xgb_best = evaluate_forecast(y_test_feat, forecast_xgb_best)

# Print
xgb_best

{'MSE': 5.402082448711131e-05,
 'RMSE': 0.0073498860186475895,
 'RMSLE': 0.0071485585395003005}

### Predicting on the actual test set here

In [131]:
df_train.shape

(2765697, 46)

In [132]:
df_test.shape

(28512, 47)

In [127]:
len(X_train_features)

45

In [133]:
# X train features
X_train_features = X_train_feat.columns.to_list()

# Values to forecast
X_test_to_forecast = df_test[X_train_features]

In [134]:
forecast = best_xgb_model.predict(X_test_to_forecast)

In [147]:
df_test['sales'] = forecast

In [148]:
df_test.head(4)

,sales,onpromotion,day_of_week,month,year,oil_price,cluster_1,cluster_2,cluster_3,cluster_4,...,holiday_Terremoto Manabi+4,holiday_Terremoto Manabi+5,holiday_Traslado Primer dia del ano,familycluster_0,familycluster_1,familycluster_2,familycluster_3,familycluster_4,familycluster_5,predictions
0,0.001531,0.000000,0.333333,0.0,0.0,0.319392,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.001531
1,0.000656,0.000000,0.333333,0.0,0.0,0.319392,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.000656
2,0.007148,0.003096,0.333333,0.0,0.0,0.319392,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.007148
3,0.021351,0.030960,0.333333,0.0,0.0,0.319392,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.021351


In [157]:
df_test = df_test.drop(columns='predictions')

In [158]:
df_test.head(3)

,sales,onpromotion,day_of_week,month,year,oil_price,cluster_1,cluster_2,cluster_3,cluster_4,...,holiday_Terremoto Manabi+3,holiday_Terremoto Manabi+4,holiday_Terremoto Manabi+5,holiday_Traslado Primer dia del ano,familycluster_0,familycluster_1,familycluster_2,familycluster_3,familycluster_4,familycluster_5
0,0.001531,0.000000,0.333333,0.0,0.0,0.319392,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.000656,0.000000,0.333333,0.0,0.0,0.319392,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.007148,0.003096,0.333333,0.0,0.0,0.319392,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [143]:
print(df_test_numerical)

None


In [159]:
unscaled = scaler.inverse_transform(df_test)

In [161]:
unscaled_df = pd.DataFrame(unscaled)

In [162]:
unscaled_df.head()

,0,1,2,3,4,5,6,7,8,9,...,36,37,38,39,40,41,42,43,44,45
0,0.001531,0.0,2.0,8.0,2017.0,46.8,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.000656,0.0,2.0,8.0,2017.0,46.8,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.007148,2.0,2.0,8.0,2017.0,46.8,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.021351,20.0,2.0,8.0,2017.0,46.8,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.001531,0.0,2.0,8.0,2017.0,46.8,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


## Going to try to manually undo scaling, need to know the min and max sales from the train data.

In [69]:
df_train_noscale = pd.read_csv(csv_train_path)

In [73]:
y_max = max(df_train_noscale['sales'])

In [74]:
y_min = min(df_train_noscale['sales'])

In [ ]:
pred_unscaled = 